El siguiente .ipynb se usa para obtener la población sobre la cual se realizará el análisis, es decir, se obtienen jugadores de un determinado rango y luego se obtienen sus historiales de partidas.

Se presentan 2 bloques, uno para las ladders de Hierro hasta Diamante, y otro para los 250 GM más altos y toda la ladder de challenger


In [ ]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()

division = 'I'
tier = 'DIAMOND'
page = '1'
ladder_csv = 'ladder_apex_csv.csv'
api_key = os.getenv('RIOT_API_KEY')

El siguiente bloque es para rangos NO APEX

In [ ]:

def get_population(tier, division, page, api_key):
    link = f'https://la1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{tier}/{division}?page={page}&api_key={api_key}'
    response = requests.get(link)
    leaguev4 = response.json()

    ladder = [summ['puuid'] for summ in leaguev4]

    return ladder

El siguiente es para rangos APEX, específicamente challenger (200) y los 250 GM con más PL

In [ ]:
from functions import get_gm_ladder, get_chall_ladder

gm_ladder = get_gm_ladder(api_key=api_key)
ch_ladder = get_chall_ladder(api_key=api_key)

apex_ladder = ch_ladder + gm_ladder
print(len(apex_ladder))

df_ladder = pd.DataFrame(data = apex_ladder, columns=['puuid_apex'])

Dado que la Ladder no es constante a lo largo del tiempo, se guarda en un CSV par asegurar un control

In [ ]:
df_ladder.to_csv(ladder_csv, index=False)
df_ladder.tail()

El siguiente bloque carga el .csv y lo convierte en una lista que contiene todos los PUUID de los jugadores APEX seleccionados

In [ ]:
df_ladder = pd.read_csv('ladder_apex_csv.csv')  
ladder_list = list(df_ladder['puuid_apex'])
ladder_list

El siguiente bloque pasa un puuid a la vez y recopila "$start - count$" partidas de cada jugador. Esto nos dará la muestra para el modelo.
NOTA: Para una muestra de 204 PUUID y un time.sleep(1.2) tuvo una duración total de 27.54mins, proceder con precaución y tiempo de sobra.

In [ ]:
matchtype = 'ranked'
queueid = '420' #Default para SoloQ
start = '15'
count = '45'
api_key = os.getenv('RIOT_API_KEY')
csv_path = 'sample_csv.csv'

In [ ]:
from functions import get_matchids, get_df_data
import time

all_data = []
seen_matches = set()
all_match_ids = []

# matchid de cada jugador en ladder_list
for summ in ladder_list:
    matches = list(set(get_matchids(
        puuid=summ,
        queueid=queueid,
        matchtype=matchtype,
        start=start,
        count=count,
        api_key=api_key
    )))
    
    new_matches = [m for m in matches if m not in seen_matches]
    if not new_matches:
        continue

    # csv temporal
    data_sample = get_df_data(history=new_matches, puuid=summ, api_key=api_key, csv_path="temp.csv")
    
    # no repetidos
    all_data.extend(data_sample)
    seen_matches.update(new_matches)
    
    time.sleep(1.7)

df_sample = pd.DataFrame(all_data)
df_sample.drop_duplicates(subset=["match_id"], inplace=True)

df_sample.to_csv("sample_apex_csv.csv", index=False)


El siguiente bloque enriquece el .csv anterior, agregando 3 columnas nuevas, asosociadas a la primera sangre de torre, de drake y de heraldo.

In [ ]:
df = pd.read_csv('sample_apex_csv.csv')
api_key = os.getenv('RIOT_API_KEY')

In [ ]:
import requests
import pandas as pd
import time
from functions import get_tower_fb, get_drake_fb, get_herald_fb

def enrich_with_objectives(df, api_key):
    """
    Enriquecer un DataFrame con primeras torres, dragones y heraldos.
    Agrega 3 columnas: tower_fb, drake_fb, herald_fb
    """
    objectives_data = []

    for match_id in df["match_id"]:
        try:
            # Obtener timeline
            timeline_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}/timeline?api_key={api_key}"
            timeline_data = requests.get(timeline_url).json()

            tower_fb = get_tower_fb(timeline_data)
            drake_fb = get_drake_fb(timeline_data)
            herald_fb = get_herald_fb(timeline_data)

            objectives_data.append({
                "match_id": match_id,
                "tower_fb": tower_fb,
                "drake_fb": drake_fb,
                "herald_fb": herald_fb
            })

            time.sleep(1.2)  # evitar rate limit

        except Exception as e:
            print(f"[{match_id}] Error en enrich_with_objectives: {e}")
            objectives_data.append({
                "match_id": match_id,
                "tower_fb": None,
                "drake_fb": None,
                "herald_fb": None
            })

    # Convertir en DataFrame
    df_obj = pd.DataFrame(objectives_data)

    # Merge con el original
    df_merged = df.merge(df_obj, on="match_id", how="left")

    # Reordenar columnas: insertar después de 'my_team_ahead_kills' y antes de 'my_team_win'
    cols = list(df_merged.columns)
    insert_at = cols.index("my_team_win")  # antes de my_team_win
    new_cols_order = (
        cols[:insert_at] +
        ["tower_fb", "drake_fb", "herald_fb"] +
        cols[insert_at:]
    )

    df_final = df_merged[new_cols_order]

    return df_final

In [ ]:
df_enriched = enrich_with_objectives(df = df, api_key=api_key)

In [ ]:
df_enriched.to_csv('sample_apex_enriched_csv.csv', index = False)